In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_cirrhosis/GSE66843'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A cell-based model unravels drivers for hepatocarcinogenesis and targets for clinical chemoprevention"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['time post infection: Day 3 post infection', 'time post infection: Day 7 post infection', 'time post infection: Day 10 post infection'], 1: ['infection: Mock infection (control)', 'infection: HCV Jc1 infection'], 2: ['cell line: Huh7.5.1']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True

def convert_trait(value):
    result = value.split(":")[1].strip().lower()
    if result == 'hcv jc1 infection':
        return 1
    elif result == 'mock infection (control)':
        return 0
    return None

def convert_age(value):
    # There is no 'age' information in the provided dictionary.
    return None

def convert_gender(value):
    # There is no 'gender' information in the provided dictionary.
    return None

# Save cohort information
save_cohort_info('GSE66843', './preprocessed/Liver_cirrhosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Liver_cirrhosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Liver_cirrhosis/trait_data/GSE66843.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
